In [ ]:
import torch

In [1]:
# import copy
import torch.nn as nn
import torch

# import copy
import torch.nn as nn
import torch

class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        # Реализуйте в этом месте конструктор.
        self.weight = weight
        self.bias = bias
        self.eps = eps
        self.momentum = momentum
        self.ema_mean_x = torch.zeros(1)
        self.ema_std_x = torch.ones(1)
        self.__valid=False
        
    def __call__(self, x):
        # normed_tensor = # Напишите в этом месте нормирование входного тензора.
        if self.__valid==True:
            mean_x = self.ema_mean_x.clone()
            std_x = self.ema_std_x.clone()
        elif self.__valid==False:
            mean_x = torch.mean(x, dim=0, keepdim=True)
            std_x = torch.mean((x - mean_x) ** 2, dim=0, keepdim=True)
            n=x.size()[0]
            self.update_running_variables(mean_x,std_x,n)

        stddev_x = torch.sqrt(std_x+self.eps)
        standard_x = (x - mean_x) / stddev_x

        return self.weight * standard_x + self.bias
        
    def eval(self):
        self.__valid=True


    def update_running_variables(self, mean_x, std_x, n):

        # тк при train счиается смещенная дисперсия то при ema пересчитываем на НЕ смещенную
        std_x_bias=((std_x*n)/(n-1)).clone() # НЕсмещеная дисперсия

        self.ema_mean_x = self.momentum * mean_x + \
                          (1.0 - self.momentum) * self.ema_mean_x
        self.ema_std_x = self.momentum * std_x_bias + \
                         (1.0 - self.momentum) * self.ema_std_x


In [2]:
input_size = 3
batch_size = 5
eps = 1e-1

batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

custom_batch_norm1d = CustomBatchNorm1d(batch_norm.weight.data,
                                        batch_norm.bias.data, eps, batch_norm.momentum)

In [7]:
weight_data=torch.tensor([ 0.2961,  1.2503, -0.1758])
batch_norm.bias.data
batch_norm.momentum

0.5

In [3]:
batch_norm.weight.data

tensor([ 0.2961,  1.2503, -0.1758])

In [3]:
torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
torch_input

tensor([[-0.3933,  1.7163,  0.6732],
        [ 0.4438,  0.3485,  0.5350],
        [ 0.8313, -0.5259, -0.0975],
        [-1.0934, -1.0620, -0.5364],
        [-0.7662,  1.2324, -0.7653]])

In [4]:
norm_output=custom_batch_norm1d(torch_input)
norm_output

tensor([[-0.0789, -0.0691,  0.9525],
        [ 0.3176, -0.4142,  1.0676],
        [ 0.5012, -0.6347,  1.5947],
        [-0.4105, -0.7700,  1.9605],
        [-0.2555, -0.1912,  2.1512]])

In [5]:
custom_output=batch_norm(torch_input)
custom_output

tensor([[-0.0789, -0.0691,  0.9525],
        [ 0.3176, -0.4142,  1.0676],
        [ 0.5012, -0.6347,  1.5947],
        [-0.4105, -0.7700,  1.9605],
        [-0.2555, -0.1912,  2.1512]], grad_fn=<NativeBatchNormBackward0>)

### Eval

In [6]:
batch_norm.eval()
custom_batch_norm1d.eval()

In [7]:
norm_output=custom_batch_norm1d(torch_input)
norm_output

tensor([[-0.1002, -0.0407,  1.1263],
        [ 0.2255, -0.3727,  1.2099],
        [ 0.3762, -0.5850,  1.5927],
        [-0.3726, -0.7151,  1.8584],
        [-0.2453, -0.1582,  1.9969]])

In [8]:
custom_output=batch_norm(torch_input)
custom_output

tensor([[-0.1002, -0.0407,  1.1263],
        [ 0.2255, -0.3727,  1.2099],
        [ 0.3762, -0.5850,  1.5927],
        [-0.3726, -0.7151,  1.8584],
        [-0.2453, -0.1582,  1.9969]], grad_fn=<NativeBatchNormBackward0>)

In [ ]:
torch.allclose(norm_output, custom_output)

In [ ]:
all_correct = True

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    
    all_correct &= torch.allclose(norm_output, custom_output) \
        and norm_output.shape == custom_output.shape
    print(all_correct)


In [ ]:
torch.allclose(norm_output, custom_output)

In [ ]:
batch_norm.eval()
custom_batch_norm1d.eval()

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output) \
        and norm_output.shape == custom_output.shape
    print(all_correct)
print(all_correct)

In [ ]:
def custom_batch_norm1d(x, weight, bias, eps):
    num_examples = x.shape[0]
    train=True
    # if train:
        mean_x = torch.mean(x, dim=0, keepdim=True)
        var_x = torch.mean((x - mean_x) ** 2, dim=0, keepdim=True)
        # update_running_variables()
    # else:
    #     self.mean_x = self.running_mean_x.copy()
    #     self.var_x = self.running_var_x.copy()

    var_x += eps
    stddev_x = torch.sqrt(var_x)
    x_minus_mean = x - mean_x
    standard_x = x_minus_mean / stddev_x

    gamma=weight#torch.ones(x.size()[1])
    #bias= torch.zeros(x.size()[1])
    return gamma * standard_x + bias